In [1]:
import pandas as pd
import mlrose
import numpy as np

# 참고 사이트: https://towardsdatascience.com/solving-travelling-salesperson-problems-with-python-5de7e883d847

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/externals/six.py:28: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  warnings.warn("The module is deprecated in version 0.21 and will be removed "


In [2]:
on_79 = pd.read_csv("point_79_on.csv")
on_79.head()
# 출근시간 가장 많이 탑승하는 데이터 가지고 분석
# 지금은 이렇게 하지만, 추후 출근-퇴근 둘을 섞어서 분석할 예정
# 아닌가? 다람쥐버스면 차라리 한곳을 지나는버스가 더 나을수도
# 디익스트라 사용하는게 출퇴근 둘다 사용하고, 이런 다람쥐버스는 그냥 한 지역 도는게 나을수도 있겠다.
# 조금 더 고민해볼 예정

,Unnamed: 0,이비카드정류장ID,정류장이름,위도,경도,횟수
0,316,4197607.0,서동탄역신일해피트리(중),37.199500,127.052483,317.0
1,808,4197590.0,솔빛마을.서해.쌍용아파트,37.195817,127.069633,304.0
2,1143,4116691.0,안녕농협삼거리,37.205217,127.014700,335.0
3,1649,4117227.0,수원대학교,37.213750,126.980150,506.0
4,2030,4197467.0,한빛마을(중),37.207567,127.069200,381.0


In [3]:
#화성,병점권에 있는 정류장들만 선별
un_hsbj = on_79[on_79['경도']<127].index
on_79 = on_79.drop(un_hsbj)
on_79 = on_79.reset_index(drop=True)

In [4]:
subset = on_79[['위도', '경도']]
tuple_on79 = [tuple(x) for x in subset.to_numpy()]

# 위도 경도 정보를 튜플형태 리스트로

In [5]:
fitness_coords = mlrose.TravellingSales(coords = tuple_on79)

In [6]:
problem_fit = mlrose.TSPOpt(length=42, fitness_fn = fitness_coords, maximize=False)

In [7]:
best_state, best_fitness = mlrose.genetic_alg(problem_fit, random_state = 2)

print('The best state found is: \n', best_state)
# the order of visiting, the number is the index of the dataframe
print('The fitness at the best state is:\n ', best_fitness)

The best state found is: 
 [35 20  7  2 22 18  9 11  8 23 15 39 10 36  5 21 30  1 14 41 17 24  3 12
 29 28 34 32  4 19 26 40 31 33 38  6 25  0 37 16 13 27]
The fitness at the best state is:
  1.0996944620870233


In [8]:
import folium 
import folium.plugins

# folium for visualization

In [9]:
latitude = 37.136881
longitude = 127.036171

# 화성시 위도경도

In [10]:
m = folium.Map(location=[latitude, longitude], zoom_start=12)

In [11]:
# best_state: index 순서
# on_79 df 사용해서 folium에 길 그리기

# reindex using the tsp result
new_index = best_state
replaced = on_79.reindex(new_index)
replaced.head()

,Unnamed: 0,이비카드정류장ID,정류장이름,위도,경도,횟수
35,30092,4100049.0,홈플러스.벌말초교,37.203300,127.038133,476.0
20,18322,4108037.0,신영통현대타운.두산위브,37.235333,127.062100,1564.0
7,4345,4197554.0,쌍용예가,37.193500,127.071800,328.0
2,1143,4116691.0,안녕농협삼거리,37.205217,127.014700,335.0
22,19885,4130136.0,남수원현대아파트,37.207667,127.010517,350.0


In [12]:
replaced.head()
# replaced dataframe은 TSP 알고리즘에서 방문 순서대로

,Unnamed: 0,이비카드정류장ID,정류장이름,위도,경도,횟수
35,30092,4100049.0,홈플러스.벌말초교,37.203300,127.038133,476.0
20,18322,4108037.0,신영통현대타운.두산위브,37.235333,127.062100,1564.0
7,4345,4197554.0,쌍용예가,37.193500,127.071800,328.0
2,1143,4116691.0,안녕농협삼거리,37.205217,127.014700,335.0
22,19885,4130136.0,남수원현대아파트,37.207667,127.010517,350.0


In [13]:
# make list of longitude and latitude to draw routes in folium
locations = replaced[['위도', '경도']]
locationlist = locations.values.tolist()
len(locationlist)
# locationlist[0]

42

In [14]:
import pandas as pd
import requests
from xml.etree import ElementTree
import numpy as np
import folium
i=1

for point in range(0, len(locationlist)):
    # i: 방문순서, pop-up에 커서를 대면 순서가 나온다.
    folium.Marker(locationlist[point], i).add_to(m)
    i = i+1
m

# draw lines
# add lines
folium.PolyLine(locationlist, color="purple", weight=2.5, opacity=1).add_to(m)

In [15]:
m.save('index.html')